In [2]:
import numpy as np 
import pandas as pd 
import os
import PIL
import pathlib
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image_dataset_from_directory
import glob
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation
import matplotlib.pyplot as plt

for chest_xray, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(chest_xray, filename))

In [4]:
class_names = ['NORMAL', 'PNEUMONIA']
Train_data_dir = pathlib.Path('/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/train')
Val_data_dir = pathlib.Path('/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val')
Test_data_dir = pathlib.Path('/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/test')
image_count = len(list(Train_data_dir.glob('*/*.jpeg')))
print('num of images :',image_count)

xrays = list(Train_data_dir.glob('*/*.jpeg'))
PIL.Image.open(str(xrays[1]))

In [7]:
batch_size = 16
img_height = 127
img_width = 127
channel = 3 
train_dataset = image_dataset_from_directory(Train_data_dir,
                                             shuffle=True,
                                             batch_size=batch_size,
                                             image_size=(img_height, img_width),
                                             class_names = class_names)

print(train_dataset)

test_dataset = image_dataset_from_directory(Test_data_dir,
                                             shuffle=True,
                                             batch_size=batch_size,
                                             image_size=(img_height, img_width),
                                             class_names = class_names)
print(test_dataset)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(Val_data_dir,
                                                             shuffle=True,
                                                             batch_size=batch_size,
                                                             image_size=(img_height, img_width),
                                                             class_names = class_names)
print(val_ds)

In [9]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

In [10]:
def data_augmenter():

    data_augmentation = tf.keras.Sequential()
    data_augmentation.add(RandomFlip('horizontal'))
    data_augmentation.add(RandomRotation(0.2))

    
    return data_augmentation

In [11]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
IMG_SIZE = (img_height, img_width)
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [12]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

In [13]:
def xray_model(image_shape=IMG_SIZE, data_augmentation=data_augmenter()):
   
    
    input_shape = image_shape + (3,)
  
    base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape,
                                                   include_top=False, 
                                                   weights='imagenet') # From imageNet
    
    base_model.trainable = False 

    inputs = tf.keras.Input(shape=input_shape) 
    
   
    x = data_augmentation(inputs)
    
    x =  preprocess_input(x)
    
  
    x = base_model(x, training=False) 
    
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    
    x = tf.keras.layers.Dropout(.2)(x)
        
   
    outputs = tf.keras.layers.Dense(1)(x)
    
    model = tf.keras.Model(inputs, outputs)
    
    return model

In [14]:
data_augmentation = data_augmenter()

for image, _ in train_dataset.take(1):
    plt.figure(figsize=(10, 10))
    first_image = image[0]
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
        plt.imshow(augmented_image[0] / 255)
        plt.axis('off')

In [16]:
model2 = xray_model(IMG_SIZE, data_augmentation)
base_learning_rate = 0.0001
model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [17]:
initial_epochs = 10
history = model2.fit(train_dataset, validation_data=test_dataset, epochs=initial_epochs)

In [18]:
acc = [0.] + history.history['accuracy']
val_acc = [0.] + history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [19]:
base_model = model2.layers[4]
base_model.trainable = True

print("Number of layers in the base model: ", len(base_model.layers))

fine_tune_at = 100

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False
    
loss_function=tf.keras.losses.BinaryCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam(lr=(0.001 * base_learning_rate))

metrics=['accuracy']

model2.compile(optimizer=optimizer,
              loss=loss_function,
              metrics=['accuracy'])


In [20]:
fine_tune_epochs = 30
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model2.fit(train_dataset,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=test_dataset)

In [21]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

model_json = model2.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

model2.save_weights("model.h5")
print("Saved model to disk")

In [22]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0, 1])
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [23]:
image_batch, label_batch = val_ds.as_numpy_iterator().next()
predictions = model2.predict_on_batch(image_batch).flatten()

predictions = tf.nn.sigmoid(predictions)
predictions = tf.where(predictions < 0.5, 0, 1)

print('Predictions:\n', predictions.numpy())
print('Labels:\n', label_batch)

plt.figure(figsize=(10, 10))
for i in range(9):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image_batch[i].astype("uint8"))
  plt.title(class_names[predictions[i]])
  plt.axis("off")

In [24]:
def preprocess(images, labels):
  return tf.keras.applications.mobilenet_v2.preprocess_input(images), labels
val_ds = val_ds.map(preprocess)
print(val_ds)
score = model2.evaluate(val_ds, verbose=0)

print(score)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

In [25]:
def prepare_image(file):
    img = keras.preprocessing.image.load_img(file, target_size=(img_height, img_width))
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return img_array_expanded_dims


In [26]:
pneumonia_file = '../input/chest-xray-pneumonia/chest_xray/val/PNEUMONIA/person1946_bacteria_4874.jpeg'
print(pneumonia_file)

img_pred = prepare_image(pneumonia_file)
predictions = model2.predict(img_pred).flatten()

predictions = tf.nn.sigmoid(predictions)
predictions = tf.where(predictions < 0.5, 0, 1)

print('Predictions:\n', predictions.numpy())
print('Labels:\n', 1)

In [27]:
normal_file = '../input/chest-xray-pneumonia/chest_xray/val/NORMAL/NORMAL2-IM-1427-0001.jpeg'
print(normal_file)

img_pred = prepare_image(normal_file)
predictions = model2.predict(img_pred).flatten()
predictions = tf.nn.sigmoid(predictions)
predictions = tf.where(predictions < 0.5, 0, 1)

print('Predictions:\n', predictions.numpy())
print('Labels:\n', 0)

In [28]:
normal_file = '../input/chest-xray-pneumonia/chest_xray/val/NORMAL/NORMAL2-IM-1431-0001.jpeg'
print(normal_file)

img_pred = prepare_image(normal_file)
predictions = model2.predict(img_pred).flatten()


predictions = tf.nn.sigmoid(predictions)
predictions = tf.where(predictions < 0.5, 0, 1)

print('Predictions:\n', predictions.numpy())
print('Labels:\n', 0)

In [29]:
pneumonia_file = '../input/chest-xray-pneumonia/chest_xray/val/PNEUMONIA/person1946_bacteria_4874.jpeg'
print(pneumonia_file)

img_pred = prepare_image(normal_file)
predictions = model2.predict(img_pred).flatten()

predictions = tf.nn.sigmoid(predictions)
predictions = tf.where(predictions < 0.5, 0, 1)

print('Predictions:\n', predictions.numpy())
print('Labels:\n', 1)